In [1]:
import os


folder_path = '/home/goodone/Desktop/01_Supervised_Training/tcp_carla_data.zip/tcp_carla_data/' # TODO: Replace this with your data root directory path

png_count = 0
related_files = []

# Walk through directory and subdirectories
for root, dirs, files in os.walk(folder_path):
    if 'rgb' in root.split(os.sep):
        for file in files:
            if file.endswith('.png'):
                png_count += 1
                base_name = os.path.splitext(file)[0]

                # Construct paths for the related files in measurements and supervision
                measurements_path = os.path.join(root.replace('rgb', 'measurements'), base_name + '.json')
                supervision_path = os.path.join(root.replace('rgb', 'supervision'), base_name + '.npy')

                # Check if these files exist
                if os.path.exists(measurements_path) and os.path.exists(supervision_path):
                    related_files.append((os.path.join(root, file), measurements_path, supervision_path))

print(f"Number of .png files: {png_count}")
print("Related files:")
print(len(related_files))

Number of .png files: 472794
Related files:
472794


In [2]:
import random

# Number of elements to sample
sample_size = 100000

# Check if the list has enough elements
if len(related_files) >= sample_size:
    sampled_list = random.sample(related_files, sample_size)
else:
    print("The list doesn't contain enough elements.")


In [3]:
import json

# Assuming 'your_list' is your list
your_list = sampled_list ## your list of elements ...

# Specify the filename
filename = 'pre_dataset_100K.json'

# Write the list to a JSON file
with open(filename, 'w') as file:
    json.dump(your_list, file, indent=4)

print(f"List successfully written to {filename}")

List successfully written to pre_dataset_100K.json


In [4]:
# list_of_inputs_and_outputs_unstructured = []
import numpy as np

json_list = []
for counter, item in enumerate(sampled_list):

    data = {}

    data["image_path"] = item[0]
    data["measurements_path"] = item[1]
    data["supervision_path"] = item[2]

    

    with open(data["measurements_path"], 'r') as file:
        data["sensor_data"] = json.load(file)
        
    data["requried_outputs"] = np.load(data["supervision_path"], allow_pickle=True)

    data_dict = data["requried_outputs"].item()

    mapping = {1:"GO LEFT" , 2: "GO RIGHT" ,  3: "GO STRAIGHT" , 4: "FOLLOW THE CURRENT LANE" , 5: "CHANGE TO LANE LEFT" , 6: "CHANGE TO LANE RIGHT" }

    json_obj = {
    "id": "identity_" + str(counter),
    "conversations": [
        {
            "from": "user",
            "value": f'Picture 1: <img>{data["image_path"]}</img>\\n Given this Ego centric image, \
            now you are autonomous driving agent, now your current position(x,y) is at \
            ({round(data["sensor_data"]["x"],3)},{round(data["sensor_data"]["y"],3)}) \\n \
            Your current speed is {round(data["sensor_data"]["speed"],3)} m/s \\n \
            Your current angle is {round(data["sensor_data"]["theta"],3)}  \\n \
            Your immediate command to follow is {mapping[data["sensor_data"]["command_near"]]} and immediate target position is at \
            ({round(data["sensor_data"]["x_command_near"],3)},{round(data["sensor_data"]["y_command_near"],3)}) \\n \
            Your later target command to follow is {mapping[data["sensor_data"]["target_command"]]} and later target position is at \
            ({round(data["sensor_data"]["x_target"],3)},{round(data["sensor_data"]["y_target"],3)}) \\n \
            The current command for applying brakes is {data["sensor_data"]["should_brake"]} \\n \
            Now you have to predict following values which are throttle (also called acceleration) (0 to 1) , steer (-1 to 1) and brake (0 to 1)\\n in the json format \
            For example: the output can be this json {{ "throttle" :x, "steer" : y, "brake": z }}'
        },
        {
            "from": "assistant",
            "value": f'{{ "throttle" :{round(float(data_dict["action"][0]),3)}, "steer" : {round(float(data_dict["action"][1]),3)}, "brake": {round(float(data_dict["action"][2]),3) } }}'
        }
        ]
        }

    json_list.append(json_obj)

In [5]:
import json
import re

# Specify the filename
filename = 'carla_supervised_100K.json'

for item in json_list:
    item["conversations"][1]["value"] = item["conversations"][1]["value"] .replace('\"', "'") 
    item["conversations"][0]["value"] = re.sub(r'\s{2,}', ' ', item["conversations"][0]["value"])
    

# Write the list to a JSON file with indentation
with open(filename, 'w') as file:
    json.dump(json_list, file, indent=4)

print(f"JSON list successfully written to {filename}")

JSON list successfully written to carla_supervised_100K.json
